In [0]:
!wget https://ceb.nlm.nih.gov/proj/malaria/cell_images.zip
!unzip cell_images.zip

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout,Lambda,ZeroPadding2D
import keras as keras
from keras import backend as K
import h5py

In [0]:
def global_average_pooling(x):
    return K.mean(x, axis = (2, 3))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

In [0]:
#generate network
model = Sequential()
model.add(Conv2D(64,3,strides=3,input_shape=(128,128,3),activation='relu',data_format = "channels_last"))
model.add(MaxPooling2D(pool_size=(1,1)))
model.add(Conv2D(32,2,strides=2,activation='relu'))
model.add(Conv2D(32,2,strides=2,activation='relu'))




model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(64,3,strides=3,activation='relu'))


model.add(Lambda(global_average_pooling, output_shape=global_average_pooling_shape))

#model.add(Dense(units = 128,activation='relu'))
#model.add(Dropout(0.1))

model.add(Dense(2, activation = 'softmax', init='uniform'))
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.5, nesterov=True)
#model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())

In [0]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [0]:
from keras.preprocessing.image import ImageDataGenerator

In [0]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split= 0.1)

test_datagen = ImageDataGenerator(validation_split= 0.1)

training_set = train_datagen.flow_from_directory('cell_images',
                                                  target_size=(128, 128),
                                                  batch_size=32,
                                                  class_mode='categorical',
                                                 subset = 'training',
                                                  shuffle = True)

test_set = test_datagen.flow_from_directory('cell_images',
                                            target_size=(128, 128),
                                            batch_size=32,
                                            class_mode='categorical', 
                                            subset = 'validation',
                                            shuffle = True)


In [0]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, CSVLogger, ReduceLROnPlateau

checkpoint = ModelCheckpoint(
    './base.model',
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=False,
    period=1
)
earlystop = EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=30,
    verbose=1,
    mode='auto'
)
tensorboard = TensorBoard(
    log_dir = './logs',
    histogram_freq=0,
    batch_size=16,
    write_graph=True,
    write_grads=True,
    write_images=False,
)

csvlogger = CSVLogger(
    filename= "training_csv.log",
    separator = ",",
    append = False
)

reduce = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.1,
    patience=3,
    verbose=1, 
    mode='auto'
)

callbacks = [checkpoint,tensorboard,csvlogger,reduce]

In [0]:
history = model.fit_generator(training_set,
                    steps_per_epoch=24804//32,
                    epochs=1,
                    validation_data=test_set,
                    callbacks = callbacks,
                    validation_steps=2754/32)

In [0]:
import cv2
import numpy as np
cv2.__version__

In [0]:

def get_output_layer(model, layer_name):
    # get the symbolic outputs of each "key" layer (we gave them unique names).
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
    layer = layer_dict[layer_name]
    return layer

In [0]:
def visualize_class_activation_map(model, img_path, output_path):
        original_img = cv2.imread(img_path)
        unmodified = original_img
        width, height, _ = original_img.shape
        print(original_img.shape)
        original_img = cv2.resize(original_img, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
        print(original_img.shape)
        
        #Reshape to the network input shape (3, w, h).
        img = np.array([np.transpose(np.float32(original_img), (1, 0, 2))])
        
        #Get the 512 input weights to the softmax.
        class_weights = model.layers[-1].get_weights()[0]
        final_conv_layer = get_output_layer(model, "conv2d_4")
        get_output = K.function([model.layers[0].input], [final_conv_layer.output, model.layers[-1].output])
        [conv_outputs, predictions] = get_output([img])
        conv_outputs = conv_outputs[0, :, :, :]

        #Create the class activation map.
        cam = np.zeros(dtype = np.float32, shape = conv_outputs.shape[1:3])
        for i, w in enumerate(class_weights[:, 1]):
                cam += w * conv_outputs[i, :, :]
        print('predictions', predictions)
        cam /= np.max(cam)
        cam = cv2.resize(cam, (height, width))
        heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
        heatmap[np.where(cam < 0.2)] = 0
        img = heatmap*0.5 + unmodified
        cv2.imwrite(output_path, img)

In [0]:
img_path = 'cell_images/Parasitized/C100P61ThinF_IMG_20150918_144104_cell_165.png'
out_path = "cam.jpg"
visualize_class_activation_map(model, img_path, out_path)

In [0]:
!cp cell_images/Uninfected/C100P61ThinF_IMG_20150918_145422_cell_3.png uninfected.png